# Computing Proximity to the Closest Train Station for all Rental Properties

In [1]:
import requests
import zipfile
import io
import geopandas as gpd
from openrouteservice import Client
import json
import pandas as pd
from shapely.geometry import Point
import ast
import time
import os
import sys
sys.path.append('../')
from scripts.utils import download_file, extract_zip

## Loading Train Station Data

In [3]:
url = 'https://s3.ap-southeast-2.amazonaws.com/cl-isd-prd-datashare-s3-delivery/Order_FYXPQM.zip'
output_dir = '../data/landing/train_stations'

# Checks to see if the directory exists and creates it if not
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created directory: {output_dir}")

# Defining the file directories
zip_path = f'{output_dir}/train_stations.zip'
extract_to = '../data/landing/train_stations/extracted'

# Download the ZIP file
download_file(url, zip_path)
    
# Extract the ZIP file
extract_zip(zip_path, extract_to)

# List files in the extracted directory
extracted_files = os.listdir(extract_to)
print("Files in extracted directory:", extracted_files)

Created directory: ../data/landing/train_stations
Files in extracted directory: ['Creative Commons Licence.html', 'll_gda94', 'PTV_METRO_TRAIN_STATION_b6722101-8db5-51f0-8a6f-d1e4fe805b73.html']


In [4]:
# Path to the shapefile
shapefile_path = '../data/landing/train_stations/extracted/ll_gda94/esrishape/whole_of_dataset/victoria/PTV/PTV_METRO_TRAIN_STATION.shp'

# Read the shapefile into a GeoDataFrame
train_gdf = gpd.read_file(shapefile_path)

# Display the first few rows of the GeoDataFrame
print(train_gdf.head())

# Display the columns of the GeoDataFrame
train_gdf.columns

  STOP_ID   LATITUDE                                          STOP_NAME  \
0   19970 -37.781193             Royal Park Railway Station (Parkville)   
1   19971 -37.788140  Flemington Bridge Railway Station (North Melbo...   
2   19972 -37.794267         Macaulay Railway Station (North Melbourne)   
3   19973 -37.807419   North Melbourne Railway Station (West Melbourne)   
4   19974 -37.788657        Clifton Hill Railway Station (Clifton Hill)   

    LONGITUDE TICKETZONE                                          ROUTEUSSP  \
0  144.952301          1                                            Upfield   
1  144.939323          1                                            Upfield   
2  144.936166          1                                            Upfield   
3  144.942570          1  Flemington,Sunbury,Upfield,Werribee,Williamsto...   
4  144.995417          1                                 Mernda,Hurstbridge   

                      geometry  
0  POINT (144.95230 -37.78119)  
1  POINT

Index(['STOP_ID', 'LATITUDE', 'STOP_NAME', 'LONGITUDE', 'TICKETZONE',
       'ROUTEUSSP', 'geometry'],
      dtype='object')

In [5]:
# Convert column names to lowercase
train_gdf.columns = [col.lower() for col in train_gdf.columns]

# Display the updated GeoDataFrame
train_gdf.head()

,stop_id,latitude,stop_name,longitude,ticketzone,routeussp,geometry
0,19970,-37.781193,Royal Park Railway Station (Parkville),144.952301,1,Upfield,POINT (144.95230 -37.78119)
1,19971,-37.788140,Flemington Bridge Railway Station (North Melbo...,144.939323,1,Upfield,POINT (144.93932 -37.78814)
2,19972,-37.794267,Macaulay Railway Station (North Melbourne),144.936166,1,Upfield,POINT (144.93617 -37.79427)
3,19973,-37.807419,North Melbourne Railway Station (West Melbourne),144.942570,1,"Flemington,Sunbury,Upfield,Werribee,Williamsto...",POINT (144.94257 -37.80742)
4,19974,-37.788657,Clifton Hill Railway Station (Clifton Hill),144.995417,1,"Mernda,Hurstbridge",POINT (144.99542 -37.78866)


In [6]:
# Checking longitude and latitude are within the reasonable limits for Victoria

# Latitude: Approx [-39, -34]
# Longitude: Approx [140, 150]

train_gdf.describe()

,latitude,longitude
count,220.000000,220.000000
mean,-37.852378,145.045691
std,0.140332,0.139902
min,-38.374235,144.661118
25%,-37.899626,144.961156
50%,-37.826147,145.036588
75%,-37.769623,145.121451
max,-37.579091,145.486379


## Loading Rental Data

In [784]:
# Path to your JSON file
json_file_path = '../data/landing/all_properties_metadata.json'

# Open and read the JSON file
with open(json_file_path, 'r') as file:
    rental_data = json.load(file)

# Convert the JSON data to a DataFrame
rental_df = pd.DataFrame.from_dict(rental_data, orient='index')
rental_df.head()

,name,cost_text,rooms,parking,desc,property_type,date_available,bond,property_features,coordinates
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[-37.9592116, 145.0649156]"
https://www.domain.com.au/8-42-wright-street-mckinnon-vic-3204-14493000,"8/42 Wright Street, McKinnon VIC 3204",$800,"[3 Beds, 2 Baths]",[1 Parking],Nestling into the manicured gardens of Allnutt...,Townhouse,Available Now,$3476,"[Floorboards, Built in wardrobes, Dishwasher, ...","[-37.9134094, 145.0308476]"
https://www.domain.com.au/802-1-ascot-vale-road-flemington-vic-3031-14593045,"802/1 Ascot Vale Road, Flemington VIC 3031",$500.00 per week,"[1 Bed, 1 Bath]",[1 Parking],"Positioned for absolute convenience, close to ...",Apartment / Unit / Flat,Available Now,$2173,"[Car Accom: Basement Carspace, Heating: Revers...","[-37.7867825, 144.9213168]"
https://www.domain.com.au/2209-22-dorcas-street-southbank-vic-3006-17158112,"2209/22 Dorcas Street, Southbank VIC 3006",$580 per week,"[1 Bed, 1 Bath]",[1 Parking],* Unverified feature,Apartment / Unit / Flat,Available Now,$2520,"[Internal Laundry*, Furnished*, Built in wardr...","[-37.8301483, 144.9696742]"
https://www.domain.com.au/511-228-a-beckett-st-melbourne-vic-3000-17114517,"511/228 A'Beckett St, Melbourne VIC 3000",$480 weekly,"[1 Bed, 1 Bath]",[− Parking],* Unverified feature,Apartment / Unit / Flat,Available Now,$2086,"[Heating*, Air conditioning, Alarm System, Int...","[-37.8102641, 144.9566396]"


In [773]:
# Checking longitude and latitude are within the reasonable limits for Victoria

# Latitude: Approx [-39, -34]
# Longitude: Approx [140, 150]
rental_df.describe()

,name,cost_text,rooms,parking,desc,property_type,date_available,bond,property_features,coordinates
count,13515,13515,13515,13515,13515,13515,8446,12543,13510,13510
unique,13352,3077,44,16,3331,16,1,991,7691,11569
top,Brighton VIC 3186,$550,"[2 Beds, 1 Bath]",[1 Parking],* Unverified feature,House,Available Now,$2390,[],"[-37.8183759, 144.9554361]"
freq,8,189,2662,5380,8817,6599,8446,583,1808,21


## The following Code should be done in Earlier Preprocessing Steps

In [764]:

# Create two new columns for latitude and longitude, but handle cases where coordinates might not be a valid list
def extract_latitude(coordinate):
    if isinstance(coordinate, list) and len(coordinate) == 2:
        return coordinate[0]  # Latitude is the first element
    else:
        return None  # Return None if coordinates are invalid

def extract_longitude(coordinate):
    if isinstance(coordinate, list) and len(coordinate) == 2:
        return coordinate[1]  # Longitude is the second element
    else:
        return None  # Return None if coordinates are invalid

# Apply the functions to extract latitude and longitude
rental_df['latitude'] = rental_df['coordinates'].apply(extract_latitude)
rental_df['longitude'] = rental_df['coordinates'].apply(extract_longitude)

# Convert latitude and longitude columns to numeric, coercing errors to NaN
rental_df['latitude'] = pd.to_numeric(rental_df['latitude'], errors='coerce')
rental_df['longitude'] = pd.to_numeric(rental_df['longitude'], errors='coerce')

# Now you can filter based on latitude and longitude
lat_less_than_140 = rental_df[rental_df['latitude'] < 140]
long_greater_than_neg34 = rental_df[rental_df['longitude'] > -34]


In [765]:
# Display the results
print("Entries with latitude less than 140:")
lat_less_than_140

Entries with latitude less than 140:


,name,cost_text,rooms,parking,desc,property_type,date_available,bond,property_features,coordinates,longitude,latitude
https://www.domain.com.au/16-rodney-court-skye-vic-3977-17196743,"16 Rodney Court, Skye VIC 3977",$530 pw,"[4 Beds, 2 Baths]",[1 Parking],* Unverified feature,House,NaN,$2303,[Heating*],"[0, 0]",0.0,0.0
https://www.domain.com.au/1510-5-joseph-road-footscray-vic-3011-17195847,"1510/5 Joseph Road, Footscray VIC 3011",$475,"[1 Bed, 1 Bath]",[− Parking],"This apartment offers a contemporary, stylish ...",Apartment / Unit / Flat,Available Now,$2064,"[Gym, Intercom, Study, Floorboards, Built in w...","[0, 0]",0.0,0.0


In [766]:
print("\nEntries with longitude greater than -34:")
long_greater_than_neg34


Entries with longitude greater than -34:


,name,cost_text,rooms,parking,desc,property_type,date_available,bond,property_features,coordinates,longitude,latitude
https://www.domain.com.au/16-rodney-court-skye-vic-3977-17196743,"16 Rodney Court, Skye VIC 3977",$530 pw,"[4 Beds, 2 Baths]",[1 Parking],* Unverified feature,House,NaN,$2303,[Heating*],"[0, 0]",0.0,0.0
https://www.domain.com.au/1510-5-joseph-road-footscray-vic-3011-17195847,"1510/5 Joseph Road, Footscray VIC 3011",$475,"[1 Bed, 1 Bath]",[− Parking],"This apartment offers a contemporary, stylish ...",Apartment / Unit / Flat,Available Now,$2064,"[Gym, Intercom, Study, Floorboards, Built in w...","[0, 0]",0.0,0.0


- 2 properties which have [0, 0] coordinates
- I think these should have already been removed via earlier preprocessing (will talk to Laura)

## Converting Rental Dataframe to Geopandas dataframe

In [785]:
# Convert string representations of lists to actual lists
rental_df['coordinates'] = rental_df['coordinates'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extract longitude and latitude from list
rental_df['latitude'] = rental_df['coordinates'].apply(lambda x: float(x[0]) if isinstance(x, (list, tuple)) and len(x) > 0 else None)
rental_df['longitude'] = rental_df['coordinates'].apply(lambda x: float(x[1]) if isinstance(x, (list, tuple)) and len(x) > 1 else None)

In [786]:
rental_df.head()

,name,cost_text,rooms,parking,desc,property_type,date_available,bond,property_features,coordinates,latitude,longitude
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[-37.9592116, 145.0649156]",-37.959212,145.064916
https://www.domain.com.au/8-42-wright-street-mckinnon-vic-3204-14493000,"8/42 Wright Street, McKinnon VIC 3204",$800,"[3 Beds, 2 Baths]",[1 Parking],Nestling into the manicured gardens of Allnutt...,Townhouse,Available Now,$3476,"[Floorboards, Built in wardrobes, Dishwasher, ...","[-37.9134094, 145.0308476]",-37.913409,145.030848
https://www.domain.com.au/802-1-ascot-vale-road-flemington-vic-3031-14593045,"802/1 Ascot Vale Road, Flemington VIC 3031",$500.00 per week,"[1 Bed, 1 Bath]",[1 Parking],"Positioned for absolute convenience, close to ...",Apartment / Unit / Flat,Available Now,$2173,"[Car Accom: Basement Carspace, Heating: Revers...","[-37.7867825, 144.9213168]",-37.786783,144.921317
https://www.domain.com.au/2209-22-dorcas-street-southbank-vic-3006-17158112,"2209/22 Dorcas Street, Southbank VIC 3006",$580 per week,"[1 Bed, 1 Bath]",[1 Parking],* Unverified feature,Apartment / Unit / Flat,Available Now,$2520,"[Internal Laundry*, Furnished*, Built in wardr...","[-37.8301483, 144.9696742]",-37.830148,144.969674
https://www.domain.com.au/511-228-a-beckett-st-melbourne-vic-3000-17114517,"511/228 A'Beckett St, Melbourne VIC 3000",$480 weekly,"[1 Bed, 1 Bath]",[− Parking],* Unverified feature,Apartment / Unit / Flat,Available Now,$2086,"[Heating*, Air conditioning, Alarm System, Int...","[-37.8102641, 144.9566396]",-37.810264,144.956640


In [787]:
# Create a GeoDataFrame
geometry = [Point(xy) for xy in zip(rental_df['longitude'], rental_df['latitude'])]
rental_gdf = gpd.GeoDataFrame(rental_df, geometry=geometry)

# Set the Coordinate Reference System (CRS) if known, e.g., EPSG:4326 for WGS84
rental_gdf.set_crs(epsg=4326, inplace=True)

# Display the first few rows of the GeoDataFrame
rental_gdf.head()

,name,cost_text,rooms,parking,desc,property_type,date_available,bond,property_features,coordinates,latitude,longitude,geometry
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[-37.9592116, 145.0649156]",-37.959212,145.064916,POINT (145.06492 -37.95921)
https://www.domain.com.au/8-42-wright-street-mckinnon-vic-3204-14493000,"8/42 Wright Street, McKinnon VIC 3204",$800,"[3 Beds, 2 Baths]",[1 Parking],Nestling into the manicured gardens of Allnutt...,Townhouse,Available Now,$3476,"[Floorboards, Built in wardrobes, Dishwasher, ...","[-37.9134094, 145.0308476]",-37.913409,145.030848,POINT (145.03085 -37.91341)
https://www.domain.com.au/802-1-ascot-vale-road-flemington-vic-3031-14593045,"802/1 Ascot Vale Road, Flemington VIC 3031",$500.00 per week,"[1 Bed, 1 Bath]",[1 Parking],"Positioned for absolute convenience, close to ...",Apartment / Unit / Flat,Available Now,$2173,"[Car Accom: Basement Carspace, Heating: Revers...","[-37.7867825, 144.9213168]",-37.786783,144.921317,POINT (144.92132 -37.78678)
https://www.domain.com.au/2209-22-dorcas-street-southbank-vic-3006-17158112,"2209/22 Dorcas Street, Southbank VIC 3006",$580 per week,"[1 Bed, 1 Bath]",[1 Parking],* Unverified feature,Apartment / Unit / Flat,Available Now,$2520,"[Internal Laundry*, Furnished*, Built in wardr...","[-37.8301483, 144.9696742]",-37.830148,144.969674,POINT (144.96967 -37.83015)
https://www.domain.com.au/511-228-a-beckett-st-melbourne-vic-3000-17114517,"511/228 A'Beckett St, Melbourne VIC 3000",$480 weekly,"[1 Bed, 1 Bath]",[− Parking],* Unverified feature,Apartment / Unit / Flat,Available Now,$2086,"[Heating*, Air conditioning, Alarm System, Int...","[-37.8102641, 144.9566396]",-37.810264,144.956640,POINT (144.95664 -37.81026)


## Calculating distances to closest train station

In [740]:
# Make sure CRS is EPSG 4326 for Rental Data
rental_gdf = rental_gdf.to_crs(epsg=4326)
print(rental_gdf.crs)

EPSG:4326


In [741]:
# Make sure CRS is EPSG 4326 for Train Data
train_gdf = train_gdf.to_crs(epsg=4326)
print(train_gdf.crs)

EPSG:4326


In [742]:
# Initialize OpenRouteService client with your API key
api_key = '5b3ce3597851110001cf62483425e548d4ed4c2ca4a6b5814be96094'
client = Client(key=api_key)

## Creating Train Station Isochrones

In [702]:
# THIS TAKES A WHILE. EXPECT IT TO BE ABOUT 13 MINUTES

def generate_isochrones(train_gdf, client, processed_stations, output_file, interval=600):  # 600 seconds = 10 minutes
    isochrone_features = []  # To store all isochrone features
    
    for index, train_station in train_gdf.iterrows():
        station_id = train_station['stop_id']  # Use STOP_ID
        station_name = train_station['stop_name']  # Use STOP_NAME
        lat, lon = train_station.geometry.y, train_station.geometry.x
        
        # Skip if station already processed
        if station_id in processed_stations:
            continue
        
        print(f"Processing: {station_id} - {station_name} (Lat: {lat}, Lon: {lon})")
        
        # Isochrone parameters for 10 minutes (600 seconds) driving time
        params_iso = {
            'profile': 'driving-car',
            'intervals': [interval],  # Single interval of 10 minutes
            'locations': [[lon, lat]],
        }
        
        try:
            # Fetch isochrones
            isochrones = client.isochrones(**params_iso)
            
            # Convert the isochrone to a GeoJSON Feature
            for feature in isochrones['features']:
                isochrone_feature = {
                    'type': 'Feature',
                    'geometry': feature['geometry'],  # Isochrone geometry
                    'properties': {
                        'station_id': station_id,  # Use STOP_ID
                        'station_name': station_name,  # Use STOP_NAME
                        'interval': interval,  # 10-minute interval
                        'latitude': lat,
                        'longitude': lon
                    }
                }
                isochrone_features.append(isochrone_feature)
            
            # Mark station as processed
            processed_stations.append(station_id)

            # Delay to prevent hitting the rate limit
            delay = 3
            time.sleep(delay)
            
        except Exception as e:
            print(f"Error processing station {station_id} - {station_name}: {e}")
    
    # Print the count and sample of isochrone features
    print(f"Isochrone features count: {len(isochrone_features)}")
    print("Sample isochrone features:", json.dumps(isochrone_features[:1], indent=2))
    
    # Create a GeoJSON structure to hold all isochrones
    isochrone_collection = {
        'type': 'FeatureCollection',
        'features': isochrone_features
    }
    
    # Ensure the directory exists
    output_directory = os.path.dirname(output_file)
    os.makedirs(output_directory, exist_ok=True)
    
    # Save the entire collection to a single GeoJSON file
    with open(output_file, 'w') as f:
        json.dump(isochrone_collection, f, indent=2)
    
    print(f"All isochrones saved to {output_file}")

# Prepare test parameters
processed_stations = []

# Run the test
generate_isochrones(
    train_gdf, 
    client, 
    processed_stations, 
    '../data/curated/isochrones/isochrones.geojson',
    interval=600
)


Processing: 19970 - Royal Park Railway Station (Parkville) (Lat: -37.7811929725527, Lon: 144.95230120580877)
Processing: 19971 - Flemington Bridge Railway Station (North Melbourne) (Lat: -37.788139984393844, Lon: 144.93932321237276)
Processing: 19972 - Macaulay Railway Station (North Melbourne) (Lat: -37.79426700459872, Lon: 144.93616600406753)
Processing: 19973 - North Melbourne Railway Station (West Melbourne) (Lat: -37.80741897361899, Lon: 144.94257002890663)
Processing: 19974 - Clifton Hill Railway Station (Clifton Hill) (Lat: -37.78865703363607, Lon: 144.9954169601631)
Processing: 19975 - Victoria Park Railway Station (Abbotsford) (Lat: -37.79915796626723, Lon: 144.9944510689111)
Processing: 19976 - Collingwood Railway Station (Abbotsford) (Lat: -37.80452601640281, Lon: 144.99375014165992)
Processing: 19977 - North Richmond Railway Station (Richmond) (Lat: -37.8103979761548, Lon: 144.9924998535909)
Processing: 19978 - West Richmond Railway Station (Richmond) (Lat: -37.814949005848

## Creating Dataframe with only 5 Rental Properties for Testing

In [743]:
# Get the first 5 rows of the GeoDataFrame
test_rental_gdf = rental_gdf.head(5)

# Display the result
test_rental_gdf.head()

,name,cost_text,rooms,parking,desc,property_type,date_available,bond,property_features,coordinates,longitude,latitude,geometry
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[145.0649156, -37.9592116]",145.064916,-37.959212,POINT (145.06492 -37.95921)
https://www.domain.com.au/8-42-wright-street-mckinnon-vic-3204-14493000,"8/42 Wright Street, McKinnon VIC 3204",$800,"[3 Beds, 2 Baths]",[1 Parking],Nestling into the manicured gardens of Allnutt...,Townhouse,Available Now,$3476,"[Floorboards, Built in wardrobes, Dishwasher, ...","[145.0308476, -37.9134094]",145.030848,-37.913409,POINT (145.03085 -37.91341)
https://www.domain.com.au/802-1-ascot-vale-road-flemington-vic-3031-14593045,"802/1 Ascot Vale Road, Flemington VIC 3031",$500.00 per week,"[1 Bed, 1 Bath]",[1 Parking],"Positioned for absolute convenience, close to ...",Apartment / Unit / Flat,Available Now,$2173,"[Car Accom: Basement Carspace, Heating: Revers...","[144.9213168, -37.7867825]",144.921317,-37.786783,POINT (144.92132 -37.78678)
https://www.domain.com.au/2209-22-dorcas-street-southbank-vic-3006-17158112,"2209/22 Dorcas Street, Southbank VIC 3006",$580 per week,"[1 Bed, 1 Bath]",[1 Parking],* Unverified feature,Apartment / Unit / Flat,Available Now,$2520,"[Internal Laundry*, Furnished*, Built in wardr...","[144.9696742, -37.8301483]",144.969674,-37.830148,POINT (144.96967 -37.83015)
https://www.domain.com.au/511-228-a-beckett-st-melbourne-vic-3000-17114517,"511/228 A'Beckett St, Melbourne VIC 3000",$480 weekly,"[1 Bed, 1 Bath]",[− Parking],* Unverified feature,Apartment / Unit / Flat,Available Now,$2086,"[Heating*, Air conditioning, Alarm System, Int...","[144.9566396, -37.8102641]",144.956640,-37.810264,POINT (144.95664 -37.81026)


In [744]:
isochrone_gdf = gpd.read_file('../data/curated/isochrones/isochrones.geojson')

# Display the first few rows to verify
isochrone_gdf.head()

,station_id,station_name,interval,latitude,longitude,geometry
0,19970,Royal Park Railway Station (Parkville),600,-37.781193,144.952301,"POLYGON ((144.89721 -37.76938, 144.89832 -37.7..."
1,19971,Flemington Bridge Railway Station (North Melbo...,600,-37.788140,144.939323,"POLYGON ((144.8873 -37.77075, 144.8921 -37.774..."
2,19972,Macaulay Railway Station (North Melbourne),600,-37.794267,144.936166,"POLYGON ((144.89347 -37.76897, 144.8941 -37.77..."
3,19973,North Melbourne Railway Station (West Melbourne),600,-37.807419,144.942570,"POLYGON ((144.90048 -37.79937, 144.89971 -37.8..."
4,19974,Clifton Hill Railway Station (Clifton Hill),600,-37.788657,144.995417,"POLYGON ((144.95551 -37.78951, 144.95568 -37.7..."


In [745]:
# Perform Spatial join to find which isochrones each Rental Property falls in
joined_gdf = gpd.sjoin(test_rental_gdf, isochrone_gdf, how='inner', predicate='within')

In [746]:
joined_gdf

,name,cost_text,rooms,parking,desc,property_type,date_available,bond,property_features,coordinates,longitude_left,latitude_left,geometry,index_right,station_id,station_name,interval,latitude_right,longitude_right
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[145.0649156, -37.9592116]",145.064916,-37.959212,POINT (145.06492 -37.95921),71,52095,Southland Railway Station (Cheltenham),600,-37.958756,145.049121
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[145.0649156, -37.9592116]",145.064916,-37.959212,POINT (145.06492 -37.95921),114,19864,Parkdale Railway Station (Parkdale),600,-37.993079,145.076327
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[145.0649156, -37.9592116]",145.064916,-37.959212,POINT (145.06492 -37.95921),115,19865,Mentone Railway Station (Mentone),600,-37.981865,145.065166
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[145.0649156, -37.9592116]",145.064916,-37.959212,POINT (145.06492 -37.95921),116,19866,Cheltenham Railway Station (Cheltenham),600,-37.966650,145.054558
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[145.0649156, -37.9592116]",145.064916,-37.959212,POINT (145.06492 -37.95921),122,19872,Highett Railway Station (Highett),600,-37.948425,145.041872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.domain.com.au/511-228-a-beckett-st-melbourne-vic-3000-17114517,"511/228 A'Beckett St, Melbourne VIC 3000",$480 weekly,"[1 Bed, 1 Bath]",[− Parking],* Unverified feature,Apartment / Unit / Flat,Available Now,$2086,"[Heating*, Air conditioning, Alarm System, Int...","[144.9566396, -37.8102641]",144.956640,-37.810264,POINT (144.95664 -37.81026),92,19842,Melbourne Central Railway Station (Melbourne C...,600,-37.809939,144.962594
https://www.domain.com.au/511-228-a-beckett-st-melbourne-vic-3000-17114517,"511/228 A'Beckett St, Melbourne VIC 3000",$480 weekly,"[1 Bed, 1 Bath]",[− Parking],* Unverified feature,Apartment / Unit / Flat,Available Now,$2086,"[Heating*, Air conditioning, Alarm System, Int...","[144.9566396, -37.8102641]",144.956640,-37.810264,POINT (144.95664 -37.81026),93,19843,Parliament Railway Station (Melbourne City),600,-37.811054,144.972911
https://www.domain.com.au/511-228-a-beckett-st-melbourne-vic-3000-17114517,"511/228 A'Beckett St, Melbourne VIC 3000",$480 weekly,"[1 Bed, 1 Bath]",[− Parking],* Unverified feature,Apartment / Unit / Flat,Available Now,$2086,"[Heating*, Air conditioning, Alarm System, Int...","[144.9566396, -37.8102641]",144.956640,-37.810264,POINT (144.95664 -37.81026),104,19854,Flinders Street Railway Station (Melbourne City),600,-37.818305,144.966964
https://www.domain.com.au/511-228-a-beckett-st-melbourne-vic-3000-17114517,"511/228 A'Beckett St, Melbourne VIC 3000",$480 weekly,"[1 Bed, 1 Bath]",[− Parking],* U

In [747]:

# Function to get distance using ORS API
def get_distance(lat1, lon1, lat2, lon2):
    # Coordinates for rental property and train station
    coordinates = [[lon1, lat1], [lon2, lat2]]
    
    try:
        # Request route
        route = client.directions(coordinates=coordinates, profile='driving-car', format='geojson')
        # Get distance in meters
        distance = route['features'][0]['properties']['segments'][0]['distance']
        return distance / 1000  # Convert to kilometers
    except Exception as e:
        print(f"Error occurred: {e}")
        return None  # Handle the case where API call fails

# Apply the function to DataFrame with a delay
def apply_distance_with_delay(row):
    distance = get_distance(row['latitude_left'], row['longitude_left'], row['latitude_right'], row['longitude_right'])
    time.sleep(2)  # Adding a delay of 1 second between requests
    return distance

# Apply with delay
joined_gdf['distance_km'] = joined_gdf.apply(apply_distance_with_delay, axis=1)


In [748]:
joined_gdf.head()

,name,cost_text,rooms,parking,desc,property_type,date_available,bond,property_features,coordinates,longitude_left,latitude_left,geometry,index_right,station_id,station_name,interval,latitude_right,longitude_right,distance_km
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[145.0649156, -37.9592116]",145.064916,-37.959212,POINT (145.06492 -37.95921),71,52095,Southland Railway Station (Cheltenham),600,-37.958756,145.049121,2.0973
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[145.0649156, -37.9592116]",145.064916,-37.959212,POINT (145.06492 -37.95921),114,19864,Parkdale Railway Station (Parkdale),600,-37.993079,145.076327,4.9081
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[145.0649156, -37.9592116]",145.064916,-37.959212,POINT (145.06492 -37.95921),115,19865,Mentone Railway Station (Mentone),600,-37.981865,145.065166,3.6822
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[145.0649156, -37.9592116]",145.064916,-37.959212,POINT (145.06492 -37.95921),116,19866,Cheltenham Railway Station (Cheltenham),600,-37.966650,145.054558,1.7517
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[145.0649156, -37.9592116]",145.064916,-37.959212,POINT (145.06492 -37.95921),122,19872,Highett Railway Station (Highett),600,-37.948425,145.041872,2.7049


In [775]:
# Group by 'name' (property name), and find the row with the minimum 'distance_km' for each group
joined_gdf_min_distance = joined_gdf.sort_values('distance_km').drop_duplicates(subset=['name'], keep='first')

# Reset index to clean up the DataFrame
joined_gdf_min_distance = joined_gdf_min_distance.reset_index(drop=True)

# Remove unecessary columns
joined_gdf_min_distance = joined_gdf_min_distance.drop(columns=['interval', 'index_right', 'latitude_right', 'longitude_right'])

# Display the updated DataFrame
joined_gdf_min_distance


,name,cost_text,rooms,parking,desc,property_type,date_available,bond,property_features,coordinates,longitude_left,latitude_left,geometry,station_id,station_name,distance_km
0,"511/228 A'Beckett St, Melbourne VIC 3000",$480 weekly,"[1 Bed, 1 Bath]",[− Parking],* Unverified feature,Apartment / Unit / Flat,Available Now,$2086,"[Heating*, Air conditioning, Alarm System, Int...","[144.9566396, -37.8102641]",144.956640,-37.810264,POINT (144.95664 -37.81026),19841,Flagstaff Railway Station (Melbourne City),0.2794
1,"802/1 Ascot Vale Road, Flemington VIC 3031",$500.00 per week,"[1 Bed, 1 Bath]",[1 Parking],"Positioned for absolute convenience, close to ...",Apartment / Unit / Flat,Available Now,$2173,"[Car Accom: Basement Carspace, Heating: Revers...","[144.9213168, -37.7867825]",144.921317,-37.786783,POINT (144.92132 -37.78678),20040,Newmarket Railway Station (Flemington),0.8620
2,"8/42 Wright Street, McKinnon VIC 3204",$800,"[3 Beds, 2 Baths]",[1 Parking],Nestling into the manicured gardens of Allnutt...,Townhouse,Available Now,$3476,"[Floorboards, Built in wardrobes, Dishwasher, ...","[145.0308476, -37.9134094]",145.030848,-37.913409,POINT (145.03085 -37.91341),19940,McKinnon Railway Station (Mckinnon),1.2109
3,"2209/22 Dorcas Street, Southbank VIC 3006",$580 per week,"[1 Bed, 1 Bath]",[1 Parking],* Unverified feature,Apartment / Unit / Flat,Available Now,$2520,"[Internal Laundry*, Furnished*, Built in wardr...","[144.9696742, -37.8301483]",144.969674,-37.830148,POINT (144.96967 -37.83015),19854,Flinders Street Railway Station (Melbourne City),1.4762
4,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[145.0649156, -37.9592116]",145.064916,-37.959212,POINT (145.06492 -37.95921),19866,Cheltenham Railway Station (Cheltenham),1.7517
